In [1]:
# !pip install openai==0.28 langchain jsonschema

In [2]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
from jsonschema import validate, ValidationError

In [3]:
# DATA SCHEMA PART

In [4]:
CUSTOMER_DATA_SCHEMA = {
    "type": "object",
    "properties": {
        "customer_id": {"type": "string"},
        "name": {"type": "string"},
        "email": {"type": "string", "format": "email"},
        "company": {"type": "string"},
        "position": {"type": "string"},
        "linkedin": {"type": "string", "format": "uri"},
        "interests": {
            "type": "array",
            "items": {"type": "string"},
        "product": {"type": "string"}
        }
    },
    "required": ["interests", "name", "email", "position"]
}

In [5]:
# Function to validate customer data against schema
def validate_customer_data(customer_data):
    try:
        validate(instance=customer_data, schema=CUSTOMER_DATA_SCHEMA)
        return True
    except ValidationError as e:
        print(f"Schema Validation Error: {e.message}")
        return False

In [6]:
# EMAIL DRAFT AND QA PART

In [7]:
# Define OpenAI model
model="gpt-3.5-turbo"

In [8]:
# Set OpenAI API key for define model
openai.api_key = "Your OpenAI API key"

In [10]:
# # To control the randomness and creativity of the generated
# text by an LLM, use temperature
llm = ChatOpenAI(openai_api_key=openai.api_key, temperature=0.7, max_tokens=500, model=model)

In [11]:
outreach_email_prompt = PromptTemplate.from_template(
    """
    You are an outreach AI responsible for drafting personalized emails for potential clients.
    Using the following customer data: {customer_data}, craft a professional and engaging email.
    Ensure the tone is appropriate, concise, and engaging.
    """
    )

In [13]:
outreach_email_chain = LLMChain(llm=llm, prompt=outreach_email_prompt)

In [14]:
qa_agent_prompt = PromptTemplate.from_template(
    """
    You are a QA agent that verifies outreach messages before sending them.
    Review the message: {draft_email}
    Ensure correctness, professionalism, and compliance with guidelines.
    Provide suggestions for improvements if necessary.
    """
)

In [15]:
qa_review_chain = LLMChain(llm=llm, prompt=qa_agent_prompt)

In [16]:
# Function to get human approval simulates human in the loop
def get_human_approval(draft_email, qa_feedback):
    print("Draft Email:")
    print(draft_email)
    print("QA Feedback:")
    print(qa_feedback)
    approval = input("Do you approve this email? (yes/no): ")
    return approval.lower() == "yes"

In [17]:
# Main function to drft mail, QA Review and approval
def generate_outreach_email(customer_data):
    if not validate_customer_data(customer_data):
        return {"error": "Invalid customer data format."}
    # Draft Outreach Email
    draft_email = outreach_email_chain.run({"customer_data": customer_data})
    
    # QA Review
    qa_feedback = qa_review_chain.run({"draft_email": draft_email})

    # Human in the Loop Review
    approved = get_human_approval(draft_email, qa_feedback)
    if not approved:
        return {"error": "Email not approved by human reviewer."}
    
    return {
        "draft_email": draft_email,
        "qa_feedback": qa_feedback,
        "approved": approved
        
    }

In [18]:
# DATA COLLECTION FROM COMPANY NAMES

In [19]:
data_gathering_prompt = PromptTemplate.from_template(
    """
    You are a data gathering AI that collects relevant personnel information from a company using public sources.
    Identify key decision-makers, managers, or other relevant personnel within the company.
    Gather publicly available details such as:
    - name
    - position
    - email (if available)
    - LinkedIn
    - Interests
    - company
    - product
    
    Structure this information in a JSON format following the defined schema. 
    Input: {company_name}
    """
)

In [20]:
data_gathering_chain = LLMChain(llm=llm, prompt=data_gathering_prompt)

In [21]:
# Function for collect company retrieval employers from open sources
def collect_customer_data(company_name):
    personnel_data = data_gathering_chain.run({"company_name": company_name})
    personnel_data_2= json.loads(personnel_data)
    personnel_data_2.pop("company")
    for i in personnel_data_2:
        customers = personnel_data_2[i]
    
    return customers

In [22]:
# IMPLEMENTATION PART

In [24]:
customer_data = collect_customer_data("Google")

In [25]:
print(customer_data)

[{'name': 'Sundar Pichai', 'position': 'CEO', 'email': 'sundar@google.com', 'LinkedIn': 'linkedin.com/in/sundarpichai', 'interests': ['technology', 'innovation', 'artificial intelligence'], 'product': 'Google Search'}, {'name': 'Ruth Porat', 'position': 'CFO', 'email': 'ruth@google.com', 'LinkedIn': 'linkedin.com/in/ruthporat', 'interests': ['finance', 'strategy', 'leadership'], 'product': 'Google Cloud'}, {'name': 'Hiroshi Lockheimer', 'position': 'SVP, Platforms & Ecosystems', 'email': 'hiroshi@google.com', 'LinkedIn': 'linkedin.com/in/hlockheimer', 'interests': ['mobile technology', 'operating systems', 'product development'], 'product': 'Android'}]


In [26]:
#Here the user can select the employee for whom the draft letter should be created or can import self customer information
#I have selected the first employee
customer_data = customer_data[0]

In [27]:
customer_data

{'name': 'Sundar Pichai',
 'position': 'CEO',
 'email': 'sundar@google.com',
 'LinkedIn': 'linkedin.com/in/sundarpichai',
 'interests': ['technology', 'innovation', 'artificial intelligence'],
 'product': 'Google Search'}

In [28]:
result = generate_outreach_email(customer_data)

Draft Email:
Subject: Exploring Collaboration Opportunities with Google Search

Dear Mr. Pichai,

I hope this email finds you well. My name is [Your Name], and I am reaching out to discuss potential collaboration opportunities with Google Search.

Having admired your leadership at Google, I am confident that our shared interests in technology, innovation, and artificial intelligence could lead to a mutually beneficial partnership. Your vision for the future of technology aligns perfectly with the innovative capabilities of Google Search.

I would love to explore how we can work together to enhance user experiences and drive advancements in the digital landscape. Your expertise and insights would be invaluable in shaping the future of our product.

I invite you to connect on LinkedIn at [LinkedIn Profile Link] to continue our conversation and explore potential synergies between our organizations.

Thank you for considering this opportunity, Mr. Pichai. I look forward to the possibility 

Do you approve this email? (yes/no):  yES


In [44]:
# The main result
result

{'draft_email': "Subject: Exploring Opportunities with Google Search\n\nDear Mr. Pichai,\n\nI hope this email finds you well. I am reaching out to express my admiration for the incredible work you have done at Google, particularly in advancing Artificial Intelligence and technology to enhance productivity.\n\nAs the CEO of Google, your leadership and vision have undoubtedly shaped the future of the tech industry. Your passion for innovation aligns perfectly with our interest in leveraging cutting-edge technology to drive productivity.\n\nI believe that Google Search, with its unparalleled capabilities, has revolutionized the way we access information. I am excited about the potential opportunities for collaboration and innovation that could arise from working with a company of Google's caliber.\n\nI would love to discuss how we can potentially partner to further enhance productivity and technological advancements. Please feel free to reach out to me at your convenience.\n\nThank you fo

In [29]:
# PART OF FOLLOW-UP 

In [30]:
follow_up_prompt = PromptTemplate.from_template(
    """
    You are an AI assistant responsible for scheduling follow-ups with potential clients.
    Based on the response from the customer, determine the best follow-up date and message.
    Customer Response: {customer_response}
    Customer Data: {customer_data}
    Provide a follow-up schedule and draft message.
    """
)

In [31]:
follow_up_chain = LLMChain(llm=llm, prompt=follow_up_prompt)

In [32]:
# The function for generate follow_uo if exist customer response
def scheduling_follow_ups(customer_data, customer_response=None):
    follow_up = None
    if customer_response:
        follow_up = follow_up_chain.run({"customer_response": customer_response, "customer_data": customer_data})
    return follow_up
    

In [33]:
follow_up =scheduling_follow_ups(customer_data, customer_response="Interested, but needs more details in two weeks.")
follow_up

'Follow-up Date: Two weeks from today\n\nDraft Message:\nSubject: Follow-up on Potential Collaboration Opportunity\n\nDear Sundar Pichai,\n\nI hope this message finds you well. I wanted to follow up on our previous conversation regarding a potential collaboration opportunity with Google Search. I understand that you are interested but require more details, and I wanted to assure you that we are excited about the possibility of working together.\n\nGiven your busy schedule as CEO of Google, I propose scheduling a follow-up meeting in two weeks to provide you with the additional information you need. Please let me know a convenient time for you, and I will make sure to accommodate it.\n\nIn the meantime, feel free to reach out if you have any questions or need further clarification on any aspect of our proposal. I look forward to the opportunity to discuss this further with you.\n\nThank you for your time and consideration.\n\nBest regards,\n\n[Your Name]\n[Your Position]\n[Your Contact 